In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd

# extracting NOC codes

url_noc = "https://www.canada.ca/en/immigration-refugees-citizenship/services/immigrate-canada/express-entry/eligibility/find-national-occupation-code.html#wb-auto-4"
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)
driver.get(url_noc)
time.sleep(5)

num_entries = driver.find_element(By.TAG_NAME, value="select")
num_entries.click()
num_options = driver.find_elements(By.TAG_NAME, value="option")
num_options[-1].click()

dict_data = {}
dict_data["TEER Category"], dict_data["NOC Code"], dict_data["Class Title"] = [[] for i in range(3)]

for i in range(6):

    cells = driver.find_elements(By.CSS_SELECTOR, value="#wb-auto-4 td")
    cells_text = [item.text for item in cells]
    dict_data["TEER Category"] += cells_text[::3]
    dict_data["NOC Code"] += cells_text[1::3]
    dict_data["Class Title"] += cells_text[2::3]

    if i <5:
        next_page = driver.find_element(By.ID, value="wb-auto-4_next")
        next_page.click()

driver.close()
df_noc = pd.DataFrame(dict_data)
df_noc.to_csv("NOC Codes.csv")
df_noc 

,TEER Category,NOC Code,Class Title
0,0,10010,Financial managers
1,0,10011,Human resources managers
2,0,10012,Purchasing managers
3,0,10019,Other administrative services managers
4,0,10020,"Insurance, real estate and financial brokerage..."
...,...,...,...
505,5,95104,Labourers in rubber and plastic products manuf...
506,5,95105,Labourers in textile processing and cutting
507,5,95106,Labourers in food and beverage processing
508,5,95107,Labourers in fish and seafood processing


In [19]:
import numpy as np

# Extracting salaries for noc codes
url = "https://www.jobbank.gc.ca/trend-analysis/search-wages"
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
driver.get(url)
input_box = driver.find_element(By.ID, value="ec-wages:wagesInput")

dict_salaries = {"Job Title":[],
                 "NOC Code":[],
                 "Low ($/hour)": [],
                 "Median ($/hour)": [],
                 "High ($/hour)": [],
                 }

time.sleep(4)
try:
    warning_close =driver.find_element(By.ID, value="j_id_48:outOfCanadaCloseBtn")
    warning_close.click()
    time.sleep(3)
except:
    print("No warning poped up")        

for i in range(len(df_noc)):
    noc_code = df_noc.loc[i]["NOC Code"]
    input_box = driver.find_element(By.ID, value="ec-wages:wagesInput")
    input_box.send_keys(Keys.CONTROL + "a")
    input_box.send_keys(Keys.DELETE)
    input_box.send_keys(noc_code)
    time.sleep(5)
    suggestion = driver.find_element(By.CLASS_NAME, value="tt-selectable")
    suggestion.click()

    if i == 0:
        search_box = driver.find_element(By.NAME, value="ec-wages:j_id_3a")
    else:
        search_box = driver.find_element(By.NAME, value="ec-wages:j_id_2z_4_n") 
        
    search_box.click()
    time.sleep(10)
    job = df_noc.loc[i]["Class Title"]
    toronto_row = driver.find_element(By.XPATH, value='//*[@id="wage-occ-report"]/tbody/tr[48]')
    toronto_cells = toronto_row.find_elements(By.TAG_NAME, value="td")

    try:
        toronto_values = [float(item.text) for item in toronto_cells[:-1]]
    except ValueError:
        print(f"{job} has no value in Toronto")
        toronto_values = [np.nan for item in toronto_cells[:-1]]

    dict_salaries["Job Title"] += [job]
    dict_salaries["NOC Code"] += [noc_code]
    dict_salaries["Low ($/hour)"] += [toronto_values[0]]
    dict_salaries["Median ($/hour)"] += [toronto_values[1]]
    dict_salaries["High ($/hour)"] += [toronto_values[2]]

No warning poped up
Underground mine service and support workers has no value in Toronto
Oil and gas well drilling and related workers and services operators has no value in Toronto
Chain saw and skidder operators has no value in Toronto
Fishing vessel deckhands has no value in Toronto
Rubber processing machine operators and related workers has no value in Toronto
Sawmill machine operators has no value in Toronto
Lumber graders and other wood processing inspectors and graders has no value in Toronto
Other wood processing machine operators has no value in Toronto
Fish and seafood plant workers has no value in Toronto
Other performers has no value in Toronto
Other support occupations in personal services has no value in Toronto
Boat and cable ferry operators and related occupations has no value in Toronto
Aquaculture and marine harvest labourers has no value in Toronto
Trappers and hunters has no value in Toronto
Mine labourers has no value in Toronto
Oil and gas drilling, servicing and 

In [22]:
df_salaries = pd.DataFrame(dict_salaries)
df_salaries.shape

(510, 5)

In [24]:
# Saving salaries data to csv

df_salaries.to_csv("salaries_in_toronto.csv")

In [19]:
# Reading data

import pandas as pd

df_salaries = pd.read_csv("salaries_in_toronto.csv")
df_salaries = df_salaries.drop(["Unnamed: 0"], axis=1)
df_salaries.head() 

,Job Title,NOC Code,Low ($/hour),Median ($/hour),High ($/hour)
0,Financial managers,10010,36.06,57.69,100.00
1,Human resources managers,10011,40.00,55.38,81.73
2,Purchasing managers,10012,38.46,62.78,84.76
3,Other administrative services managers,10019,27.50,45.19,83.07
4,"Insurance, real estate and financial brokerage...",10020,33.65,56.41,96.15


In [20]:
clean_df = df_salaries.dropna()
clean_df.shape

(432, 5)

In [23]:
# Formating the numbers
pd.options.display.float_format = '{:,.0f}'.format

# Select relevant columns from the clean dataframe
df_salaries_yearly = clean_df[["Job Title", "NOC Code"]].copy()

# Define a constant for the number of working hours in a year
HOURS_PER_YEAR = 37.5 * 52

# Convert hourly wages to yearly salaries
df_salaries_yearly[["Low ($/year)", "Median ($/year)", "High ($/year)"]] = (
    clean_df[["Low ($/hour)", "Median ($/hour)", "High ($/hour)"]] * HOURS_PER_YEAR
)

# Display the resulting dataframe
df_salaries_yearly

,Job Title,NOC Code,Low ($/year),Median ($/year),High ($/year)
0,Financial managers,10010,"70,317","112,496","195,000"
1,Human resources managers,10011,"78,000","107,991","159,374"
2,Purchasing managers,10012,"74,997","122,421","165,282"
3,Other administrative services managers,10019,"53,625","88,120","161,986"
4,"Insurance, real estate and financial brokerage...",10020,"65,618","110,000","187,492"
...,...,...,...,...,...
504,"Labourers in wood, pulp and paper processing",95103,"32,272","39,000","49,803"
505,Labourers in rubber and plastic products manuf...,95104,"32,272","36,368","56,550"
506,Labourers in textile processing and cutting,95105,"32,272","35,100","48,750"
507,Labourers in food and beverage processing,95106,"32,272","33,150","42,900"


In [33]:
# Top 10 jobs based on Median salary

# Set pandas option to display full text in cells
pd.set_option('display.max_colwidth', None)

df_salaries_yearly.sort_values(by="Median ($/year)", ascending=False)[:10][["Job Title", "Median ($/year)"]]

,Job Title,Median ($/year)
36,Managers in natural resources production and fishing,"165,438"
106,Lawyers and Quebec notaries,"126,555"
2,Purchasing managers,"122,421"
21,Fire chiefs and senior firefighting officers,"121,076"
107,University professors and lecturers,"120,003"
9,Engineering managers,"120,003"
11,Computer and information systems managers,"120,003"
54,Other professional occupations in physical sciences,"119,067"
13,Government managers - health and social policy development and program administration,"117,858"
10,Architecture and science managers,"117,000"


In [34]:
# Top 10 high potential jobs
df_salaries_yearly.sort_values(by="High ($/year)", ascending=False)[:10][["Job Title", "High ($/year)"]]

,Job Title,High ($/year)
36,Managers in natural resources production and fishing,"1,105,708"
107,University professors and lecturers,"202,508"
277,"Air pilots, flight engineers and flying instructors","195,936"
0,Financial managers,"195,000"
106,Lawyers and Quebec notaries,"193,128"
26,Corporate sales managers,"187,492"
4,"Insurance, real estate and financial brokerage managers","187,492"
45,"Securities agents, investment dealers and brokers","187,492"
11,Computer and information systems managers,"187,492"
10,Architecture and science managers,"181,077"


In [37]:
spread_col = df_salaries_yearly ["High ($/year)"] - df_salaries_yearly["Low ($/year)"]
df_salaries_yearly.insert(2, "Spread", spread_col)
df_salaries_yearly.head()

,Job Title,NOC Code,Spread,Low ($/year),Median ($/year),High ($/year)
0,Financial managers,10010,"124,683","70,317","112,496","195,000"
1,Human resources managers,10011,"81,374","78,000","107,991","159,374"
2,Purchasing managers,10012,"90,285","74,997","122,421","165,282"
3,Other administrative services managers,10019,"108,362","53,625","88,120","161,986"
4,"Insurance, real estate and financial brokerage managers",10020,"121,875","65,618","110,000","187,492"


In [36]:
# Low risk jobs
low_risk = df_salaries_yearly.sort_values(by="Spread")
low_risk[:10]

,Job Title,NOC Code,Spread,Low ($/year),Median ($/year),High ($/year)
467,Cashiers,65100,"1,852","32,272","32,272","34,125"
481,"Dry cleaning, laundry and related occupations",65320,"2,535","32,272","32,272","34,808"
445,"Inspectors and graders, textile, fabric, fur and leather products manufacturing",94133,"4,778","32,272","32,272","37,050"
493,Harvesting labourers,85101,"6,220","32,272","32,272","38,493"
443,"Weavers, knitters and other fabric making occupations",94131,"6,610","32,272","33,150","38,883"
444,Industrial sewing machine operators,94132,"6,728","32,272","32,272","39,000"
472,"Food counter attendants, kitchen helpers and related support occupations",65201,"6,728","32,272","32,272","39,000"
509,"Other labourers in processing, manufacturing and utilities",95109,"9,262","32,272","33,150","41,535"
447,"Industrial butchers and meat cutters, poultry preparers and related workers",94141,"9,750","39,000","41,340","48,750"
507,Labourers in food and beverage processing,95106,"10,628","32,272","33,150","42,900"


In [26]:
# You can modify the code for Canada wide salaries 

job = df_noc.loc[0]["Class Title"]

national_row = driver.find_element(By.CLASS_NAME, value="wage-national")
national_cells = national_row.find_elements(By.TAG_NAME, value="td")
national_values = [float(item.text) for item in national_cells[:-1]]
national_values

[33.62, 55.29, 87.91]

In [34]:
# You can modify the code for salaries in Montreal

montreal_row = driver.find_element(By.XPATH, value='//*[@id="wage-occ-report"]/tbody/tr[33]')
montreal_cells = montreal_row.find_elements(By.TAG_NAME, value="td")
montreal_values = [float(item.text) for item in montreal_cells[:-1]]
montreal_values

[32.0, 52.88, 82.42]

In [35]:
# You can modify the code for salaries in Vancouver

vancouver_row = driver.find_element(By.XPATH, value='//*[@id="wage-occ-report"]/tbody/tr[83]')
vancouver_cells = vancouver_row.find_elements(By.TAG_NAME, value="td")
vancouver_values = [float(item.text) for item in vancouver_cells[:-1]]
vancouver_values

[23.0, 50.0, 72.12]

In [36]:
# You can modify the code for salaries in Ottawa

ottawa_row = driver.find_element(By.XPATH, value='//*[@id="wage-occ-report"]/tbody/tr[46]')
ottawa_cells = ottawa_row.find_elements(By.TAG_NAME, value="td")
ottawa_values = [float(item.text) for item in ottawa_cells[:-1]]
ottawa_values

[43.59, 60.1, 92.31]